In [119]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from typing import Dict
from pyhtml2pdf import converter
import requests
import os
import re




**Scrape Documentations**

In [ ]:
technology_scraping_dict = {
    "docker-compose": {
        "url": "https://docs.docker.com/compose/compose-file/",
        "sub_urls": True,
        "scraping": {
            "html_class": "article",
            "css_class": "class",
            "css_class_name": "prose max-w-none dark:prose-invert"
        }
    },
    "docker": {
        "url": "https://docs.docker.com/reference/dockerfile/",
        "sub_urls": False,
        "scraping": {
            "html_class": "article",
            "css_class": "class",
            "css_class_name": "prose max-w-none dark:prose-invert"
        }
    },
    "spring-boot": [
        {
            "url": "https://docs.spring.io/spring-boot/appendix/application-properties/index.html",
                "sub_urls": False,
                "scraping": {
                    "html_class": "article",
                    "css_class": "class",
                    "css_class_name": "doc"
                },
        },
        {
            "url": "https://docs.spring.io/spring-boot/docs/1.0.1.RELEASE/reference/html/howto-properties-and-configuration.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "div",
                "css_class": "class",
                "css_class_name": "chapter"
            }
        }
    ],
    "maven": [
        {
            "url": "https://maven.apache.org/pom.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "main",
                "css_class": "id",
                "css_class_name": "bodyColumn"
            }
        },
        {
            "url": "https://maven.apache.org/ref/3.9.7/maven-model/maven.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "main",
                "css_class": "id",
                "css_class_name": "bodyColumn"
            }
        },
                {
            "url": "https://maven.apache.org/ref/3.9.7/maven-settings/settings.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "main",
                "css_class": "id",
                "css_class_name": "bodyColumn"
            }
        },
        {
            "url": "https://maven.apache.org/ref/3.9.7/maven-core/toolchains.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "main",
                "css_class": "id",
                "css_class_name": "bodyColumn"
            }
        }
    ],
    "nodejs": [
        {
            "url": "https://nodejs.org/docs/latest/api/packages.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "div",
                "css_class": "id",
                "css_class_name": "apicontent"
            }
        },
        {
            "url": "https://docs.npmjs.com/cli/v10/configuring-npm/package-json",
            "sub_urls": False,
            "scraping": {
                "html_class": "main",
                "css_class": "class",
                "css_class_name": "Box-sc-g0xbh4-0 jrNUvm"
            }
        },
    ],
    "tsconfig": {
        "url": "https://www.typescriptlang.org/tsconfig/",
        "sub_urls": False,
        "scraping": {
            "html_class": "main",
            "css_class": "role",
            "css_class_name": "main"
        }
    },
    "mysql": [
        {
            "url": "https://dev.mysql.com/doc/refman/8.4/en/server-system-variables.html",
            "sub_urls": False,
            "scraping": {
                "html_class": "div",
                "css_class": "id",
                "css_class_name": "docs-main-inner"
            }
        },
        {
            "url": "https://dev.mysql.com/doc/refman/8.4/en/innodb-parameters.html",
                "sub_urls": False,
                "scraping": {
                    "html_class": "div",
                    "css_class": "id",
                    "css_class_name": "docs-main-inner"
                }
        }
    ]
}

In [120]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def scrape_sub_urls(url):
    # Fetch the main page
    response = requests.get(url)
    response.raise_for_status()  # Ensures we notice if something goes wrong

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all links within the page's <a> tags
    links = soup.find_all('a', href=True)
    # Normalize and filter URLs
    sub_urls = set()
    for link in links:
        # Create a full URL if the link is relative
        full_url = urljoin(url, link['href'])
        if full_url.startswith(url):  # Optional: filter for specific domain
            sub_urls.add(full_url)
    return sub_urls

In [121]:
from xhtml2pdf import pisa
from io import BytesIO

# HTML content
html_content = """
<!DOCTYPE html>
<html>
<body>
    {}
</body>
</html>
"""


def scrape(scraping_values: Dict, tech_dir_name: str):
    main_url = scraping_values["url"]

    if scraping_values["sub_urls"]:
        sub_urls = scrape_sub_urls(main_url)

        for url in sub_urls:
            print(url)

            name = url.split("/")[-2]
            output_file = tech_dir_name + f"/{name}.pdf"

            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            main_body = soup.find(scraping_values["scraping"]["html_class"], {scraping_values["scraping"]["css_class"]: scraping_values["scraping"]["css_class_name"]})

            pdf_output = BytesIO()
            pisa.CreatePDF(html_content.format(main_body), dest=pdf_output, encoding='utf-8')

            if not os.path.exists(output_file):
                # Open a PDF file for writing in binary mode
                with open(output_file, "wb") as pdf_file:
                    # Write the PDF content to the file  
                    pdf_file.write(pdf_output.getvalue())
            else:
                print("File already exists.")
    
    else:
        print(main_url)
        name = "_".join(main_url.split("/")[-2:])
        output_file = tech_dir_name + f"/{name}.pdf"

        response = requests.get(main_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        main_body = soup.find(scraping_values["scraping"]["html_class"], {scraping_values["scraping"]["css_class"]: scraping_values["scraping"]["css_class_name"]})

        pdf_output = BytesIO()
        pisa.CreatePDF(html_content.format(main_body), dest=pdf_output, encoding='utf-8')

        if not os.path.exists(output_file):
            # Open a PDF file for writing in binary mode
            with open(output_file, "wb") as pdf_file:
                # Write the PDF content to the file  
                pdf_file.write(pdf_output.getvalue())
        else:
            print("File already exists.")



In [122]:
for technology, scraping_values in technology_scraping_dict.items():
    print("Scrape: ", technology)

    tech_dir_name = f"../../data/tech_docs/{technology}"

    os.makedirs(tech_dir_name, exist_ok=True)

    if len(os.listdir(tech_dir_name)) > 0:
        print(f"Technology {technology} already scraped.")
        continue

    if isinstance(scraping_values, list):
        for x in scraping_values:
            scrape(scraping_values=x, tech_dir_name=tech_dir_name)

    else:
        scrape(scraping_values=scraping_values, tech_dir_name=tech_dir_name)

Scrape:  docker-compose
Technology docker-compose already scraped.
Scrape:  docker
Technology docker already scraped.
Scrape:  spring-boot
Technology spring-boot already scraped.
Scrape:  maven
Technology maven already scraped.
Scrape:  nodejs
Technology nodejs already scraped.
Scrape:  tsconfig
Technology tsconfig already scraped.
Scrape:  mysql
https://dev.mysql.com/doc/refman/8.4/en/server-system-variables.html
Name:  en_server-system-variables.html
https://dev.mysql.com/doc/refman/8.4/en/innodb-parameters.html
Name:  en_innodb-parameters.html


**Scrape Stack Overflow Posts**

In [128]:
import itertools

technologies = ["docker", "docker-compose", "maven", "spring"]

combinations = list(itertools.combinations(technologies, r=2))

combinations


[('docker', 'docker-compose'),
 ('docker', 'maven'),
 ('docker', 'spring'),
 ('docker-compose', 'maven'),
 ('docker-compose', 'spring'),
 ('maven', 'spring')]

**Scrape Blog Posts**

In [73]:
import requests
import re
import backoff
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from xhtml2pdf import pisa
from io import BytesIO


# HTML content
html_content = """
<!DOCTYPE html>
<html>
<body>
    {}
</body>
</html>
"""


# List of technology pairs
technologies = ["spring-boot", "maven", "docker-compose", "docker"]
tech_pairs = list(itertools.combinations(technologies, r=2))

# Base URL for Google search
base_url = "https://www.bing.com/search?q=site:"
headers  = {'User-Agent':  UserAgent().chrome}


@backoff.on_exception(
    backoff.expo,
    requests.exceptions.RequestException,
    max_tries=8,
    jitter=None)
def get_urls(search_url: str):
    urls = []

    response = requests.get(search_url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    search_results = soup.find_all("h2")
    search_results = [result for result in search_results if "medium.com" in str(result)][:5]

    if not search_results:
        raise Exception()

    for elem in search_results:
        match = re.findall(r'"([^"]*)"', str(elem))
        link = match[1]
        urls.append(link)

    if not urls:
        raise Exception()
    
    return urls


# Iterate over each technology pair
for pair in tech_pairs:
    os.makedirs(f"../../data/blog_posts/{pair[0]}_{pair[1]}", exist_ok=True)

    print("Pair: ", pair)

    search_url = base_url + "medium.com" + " " + f"{pair[0]} {pair[1]}"

    urls = get_urls(search_url=search_url)

    print(urls)

    for url in urls:
        print(url)

        blog_post_name = "-".join(url.split("/")[-1].split("-")[0:-1])

        output_file = f"../../data/blog_posts/{pair[0]}_{pair[1]}/{blog_post_name}.pdf"

        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        article = soup.find("article")
        
        pdf_output = BytesIO()
        pisa.CreatePDF(html_content.format(article), dest=pdf_output, encoding='utf-8')

        if not os.path.exists(output_file):
            # Open a PDF file for writing in binary mode
            with open(output_file, "wb") as pdf_file:
                # Write the PDF content to the file  
                pdf_file.write(pdf_output.getvalue())
        else:
            print("File already exists.")

Pair:  ('spring-boot', 'maven')
['https://gurselgazii.medium.com/dockerizing-your-maven-spring-boot-application-a-step-by-step-guide-e267c2d9e8e1', 'https://medium.com/swlh/build-a-docker-image-using-maven-and-spring-boot-58147045a400', 'https://medium.com/@youeleven/understanding-maven-with-spring-boot-9a4e9becb1d9', 'https://medium.com/@AlexanderObregon/simplifying-java-microservices-development-with-maven-and-spring-boot-bb9a9286236e', 'https://betulsahinn.medium.com/dockerizing-a-spring-boot-application-and-using-the-jib-maven-plugin-95c329866f34']
https://gurselgazii.medium.com/dockerizing-your-maven-spring-boot-application-a-step-by-step-guide-e267c2d9e8e1


Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw,'
The src attribute of image tag is empty!
'<img alt="" class="bg lr nv c" height="700" loading="eager" role="presentation" width="700"/>'


File already exists.
https://medium.com/swlh/build-a-docker-image-using-maven-and-spring-boot-58147045a400


Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw,'
The src attribute of image tag is empty!
'<img alt="" class="bg he hf c" height="380" loading="eager" role="presentation" width="700"/>'


https://medium.com/@youeleven/understanding-maven-with-spring-boot-9a4e9becb1d9
File already exists.
https://medium.com/@AlexanderObregon/simplifying-java-microservices-development-with-maven-and-spring-boot-bb9a9286236e


...

https://medium.com/simform-engineering/setting-up-a-local-development-environment-using-docker-compose-551efb4ec0ee


Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw,'
The src attribute of image tag is empty!
'<img alt="" class="bg mu oa c" height="350" loading="eager" role="presentation" width="700"/>'
Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx

https://medium.com/@seventechnologiescloud/local-sqlserver-database-via-docker-compose-the-ultimate-guide-f1d9f0ac1354


Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx) and (max-width: 700px) 80vw, (-webkit-min-device-pixel-ratio: 2.5) and (max-width: 700px) 80vw, (min-resolution: 2dppx) and (max-width: 700px) 100vw, (-webkit-min-device-pixel-ratio: 2) and (max-width: 700px) 100vw,'
The src attribute of image tag is empty!
'<img alt="" class="bg lr mx c" height="368" loading="eager" role="presentation" width="700"/>'
Attribute 'src' must be set!
'<source data-testid="og" sizes="(min-resolution: 4dppx) and (max-width: 700px) 50vw, (-webkit-min-device-pixel-ratio: 4) and (max-width: 700px) 50vw, (min-resolution: 3dppx) and (max-width: 700px) 67vw, (-webkit-min-device-pixel-ratio: 3) and (max-width: 700px) 65vw, (min-resolution: 2.5dppx

File already exists.
